In [1]:

import torch
import torch
import numpy as np
import sys
import os
sys.path.append(os.path.join(os.path.abspath(''), 'core'))
from zeropi import ZeroPi
import general 



In [2]:
#create ZeroPi

#paramaters to pptimise over
EJ = torch.tensor(10.00, requires_grad=True, dtype=torch.double)
EL = torch.tensor(0.04, requires_grad=True, dtype=torch.double)
ECJ = torch.tensor(20, requires_grad=True, dtype=torch.double)
EC = torch.tensor(0.04, requires_grad=True, dtype=torch.double)
dEJ = torch.tensor(0.0, requires_grad=True, dtype=torch.double)
dCJ = torch.tensor(0.0, requires_grad=True, dtype=torch.double)
ECS= torch.tensor(0.1, requires_grad=True, dtype=torch.double)

#fixed params of system
flux = 0.5
ng = 0.1

#fixed params of computation
ncut = 30
hamiltonian_creation_solution = "manual_discretization_davidson"
discretization_dim = 100


zeropi = ZeroPi(EJ=EJ, 
                 EL=EL , 
                 ECJ=ECJ, 
                 EC=EC, 
                 dEJ=dEJ,
                 ECS = ECS, 
                 dCJ=dCJ, 
                 flux = flux,
                 ng=ng, 
                 ncut = ncut,
                 discretization_dim =discretization_dim,
                 hamiltonian_creation_solution=hamiltonian_creation_solution)



In [7]:
H = zeropi.manual_discretization_H()

 /Users/judd/Documents/AD_superconducting_qubit/ADOSC/core/zeropi.py: 180UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
 /Users/judd/Documents/AD_superconducting_qubit/ADOSC/core/zeropi.py: 201UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
 /Users/judd/Documents/AD_superconducting_qubit/ADOSC/core/zeropi.py: 170UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
 /Users/judd/Documents/AD_superconducting_qubit/ADOSC/core/zeropi.py: 211UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clo

In [3]:
eigvals, eigvecs = zeropi.esys()
eigvals

 /Users/judd/Documents/AD_superconducting_qubit/ADOSC/core/zeropi.py: 180UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
 /Users/judd/Documents/AD_superconducting_qubit/ADOSC/core/zeropi.py: 201UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
 /Users/judd/Documents/AD_superconducting_qubit/ADOSC/core/zeropi.py: 170UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
 /Users/judd/Documents/AD_superconducting_qubit/ADOSC/core/zeropi.py: 211UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clo

tensor([142.5768, 142.5780], dtype=torch.float64,
       grad_fn=<symeig_torchfcnBackward>)

In [4]:
t = general.t2_rate(qubit = zeropi, eigvecs=eigvecs, eigvals =eigvals, sparse = True)

 /Users/judd/Documents/AD_superconducting_qubit/ADOSC/venv/lib/python3.8/site-packages/torch_sparse/matmul.py: 97

In [5]:
t.backward()

In [6]:
print(EJ.grad)
print(EL.grad)
print(dEJ.grad)

tensor(6.0278e-25, dtype=torch.float64)
tensor(4.6853e-18, dtype=torch.float64)
tensor(2.9898e-24, dtype=torch.float64)
